## Урок 6. Видеоурок “Производная функции одной переменной”

1. Найти производную выражения:

    a.
$$
\sin x \cdot \cos x
$$

In [1]:
# Производная от произведения функций равна сумме произведений производной от 1-й на 2-ю и 1-й на производную от 2-й функций.

$$
\sin x \cdot \cos x =  (\sin x)' \cdot \cos x + \sin x \cdot (\cos x)' = \cos x \cdot \cos x + \sin x \cdot (-\sin x) = \cos^2 x - \sin^2 x 
$$

    b.
$$
\ln (2x+1)^3
$$

In [2]:
# Производная от функции от функции от функции (и т.д.) равна произведению производных этих функций.

$$
\ln (2x+1)^3 = (\ln (2x+1)^3)' \cdot ((2x+1)^3)' \cdot (2x+1)' = \frac{1}{(2x+1)^3} \cdot 3(2x+1)^2 \cdot 2 = \frac{6}{2x+1} 
$$

    c.
$$
\sqrt{\sin^2(\ln(x^3))}
$$

In [3]:
# Производная от функции от функции от функции (и т.д.) равна произведению производных этих функций.

$$
\sqrt{\sin^2(\ln(x^3))} = (\sin(\ln(x^3)))' \cdot (\ln(x^3))' \cdot (x^3)'
$$

In [5]:
# Однако если действовать правильно и не сокращать корень из квадрата синуса, то получим недифференцируемую функцию!

    d.
$$
\frac{x^4}{\ln(x)}
$$

In [4]:
# Производная от частного функций равна разнице произведений производной от 1-й на 2-ю и 1-й на производную от 2-й функций, деленной на квадрат 2-й функции.

$$
\frac{x^4}{\ln(x)} = \frac{(x^4)' \cdot \ln x - x^4 \cdot (\ln x)'}{\ln^2(x)}
$$

2. Найти выражение производной функции и ее значение в точке:
$$
f(x)=\cos(x^2+3x),x_0=\sqrt\pi
$$

3. \* Найти значение производной функции в точке:
$$
f(x)=\frac{x^3-x^2-x-1}{1+2x+3x^2-4x^3},x_0=0
$$

4. Найти угол наклона касательной к графику функции в точке:
$$
f(x)=\sqrt{3x}\cdot\ln x,x_0=1
$$